<a href="https://colab.research.google.com/github/keiohta/2020_deeprl_summer_school_lecture3/blob/master/rlss2020_lecture3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 強化学習サマースクール 第3回 Model-based RL

## 目次
1. [はじめに]()
1. [モデル予測制御]()
1. [アンサンブルモデルによるモデルベース強化学習]()

## はじめに
第3回では、モデルベースRLとしてダイナミクス（状態遷移）モデルを用いた制御手法について取り扱います。
本資料では、[Pendulum](https://gym.openai.com/envs/Pendulum-v0/)を題材に、基礎的なモデルベースRLのコードを読み書きすることで理論と実装両面の理解を進めることを目的とします。

まず初めに、今回の演習で必要なライブラリを一度にインストールしておきましょう。


In [ ]:
!apt update > /dev/null 2>&1
!apt install xvfb > /dev/null 2>&1
!pip install gym-notebook-wrapper cpprb tf2rl > /dev/null 2>&1

## モデル予測制御（MPC）

MPCと一言にまとめてもたくさん種類がありますが、本演習では下記論文で提案されているものを考えます。
- [Neural Network Dynamics for Model-Based Deep Reinforcement Learning with Model-Free Fine-Tuning](https://arxiv.org/abs/1708.02596)

上記論文では、ダイナミクスモデルを学習し、それを用いてRandom-sample Shooting (RS) という方法を用いて制御問題を解きます。

### RS: Random-sample Shooting
RSでは、ダイナミクスモデル $f$ と報酬（またはコスト）関数 $g$ が（正確でないにせよ）既知であると仮定し、
それを用いて現在の状態 $s_t \in \mathbb{R}^{N^{state}}$ における行動 $a_t$ を生成します。

ダイナミクスモデル $f$ が既知なので、次の状態を $s_{t+1} = f(s_t, a_t)$ のように予測することができます。
更に、報酬関数を用いて $r_{t+1} = g(s_t, a_t, s_{t+1})$ からその状態において実行した行動がどれくらい良いかが得られます。
このように、次の状態・報酬を反復的に $M$ 回計算することで、 $M$ ステップ後までの累積報酬和 $R_t^{t+M} = \sum_{m=1}^M r_{t+m}$が得られます。

RSでは、このアイデアを用いて**ランダムに**行動を $N^\text{episode}$ エピソード分生成し、その中で最も
累積報酬和が高かった最初の行動を選択し、1ステップ分だけエピソードを進めます。
具体的には、以下のように進めます。

1. 現在の状態を $s_t$ とし、$N^\text{episode}$エピソード分を複製する $\mathbf{s} \in \mathbb{R}^{N^\text{episode} \times N^{state}}$ 
1. $N^\text{episode}$ エピソード分のランダムな行動 $\mathbf{a} \in \mathbb{R}^{N^\text{episode} \times N^{action}}$ を生成する
1. ダイナミクスモデルを用いて次の状態と報酬を予測する
1. $T$ステップ先読みするまで2に戻って再度繰り返す
1. $N^\text{episode}$ エピソードの中で最も累積報酬が高いエピソードを選択し、その最初の行動を用いて1ステップ環境を進める

### 全体の流れ
今回扱う手法では、ダイナミクスモデルを学習しながらRSを行います。
全体的な流れは以下のようになります。

1. ランダムに遷移を収集し状態遷移モデル $f$ を事前学習
1. $N^\text{episode}$ エピソード分繰り返し
  1. エピソード終端条件を満たすまで繰り返し
    1. RSを用いて環境を進める
    1. 得られた遷移 $(s_t, a_t, s_{t+1})$ をバッファ $\mathcal{D}$ に保存
  1. ダイナミクスモデル $f$ を学習

それでは実装していきましょう。

#### 環境の実装

まず初めに今回の演習で使う環境であるOpenAI Gymの[Pendulum](https://gym.openai.com/envs/Pendulum-v0/)環境を用意します。

Pendulumは状態数が3（現在の振り子の角度と角速度：$\{\sin\theta, \cos\theta, \dot{\theta}\}$）, 行動数が1（トルク：$\tau$）の連続値（非離散値）入出力の中で最も簡単な環境の一つです。

また、Pendulumがどのような動作をするかを可視化して確認しておきましょう。
制御器を設計する前に環境の特徴を把握することは非常に重要です。

In [ ]:
import gnwrapper
import gym
 
env = gym.make("Pendulum-v0")

# 可視化用の環境。JupyterNotebookで可視化するためのラッパーをかましています
monitor_env = gnwrapper.Monitor(gym.make("Pendulum-v0"), size=(400, 300), directory='.', force=True,
                                video_callable=lambda ep: True)
episode_max_steps = 200

for episode_idx in range(3):
    monitor_env.reset()
    total_rew = 0.
    for _ in range(episode_max_steps):
        act = monitor_env.action_space.sample()
        _, rew, done, _ = monitor_env.step(act)
        total_rew += rew
        if done:
            break
    print("iter={0: 3d} total reward: {1: 4.4f}".format(episode_idx, total_rew))

monitor_env.display()

また、ここでPendulum特有の報酬関数についても実装しておきましょう。
報酬計算部分のコードを[ここ](https://github.com/openai/gym/blob/6df1b994bae791667a556e193d2a215b8a1e397a/gym/envs/classic_control/pendulum.py#L51)から抜粋して使用します。
基本的には、以下のような報酬関数になっています。

$
r = -\left( \left\| \theta_t - \theta^\text{goal}  \right\|^{2} + 0.1 \times \dot{\theta}_t^2 + 0.001 \times \tau_t^2 \right)
$

大まかには、目標角度 $\theta^\text{goal}$ に到達する際に、角速度と必要なトルクを最小化するような方策が最適な方策であることが分かります。
また、**0に近いほど良い**ことも分かります。

In [ ]:
def angle_normalize(x):
    return ((x + np.pi) % (2 * np.pi)) - np.pi
 
 
def reward_fn(obses, acts):
    is_single_input = obses.ndim == acts.ndim and acts.ndim == 1
    if is_single_input:
        thetas = np.arctan2(obses[1], obses[0])
        theta_dots = obses[2]
    else:
        assert obses.ndim == acts.ndim == 2
        assert obses.shape[0] == acts.shape[0]
        acts = np.squeeze(acts)
        thetas = np.arctan2(obses[:, 1], obses[:, 0])
        theta_dots = obses[:, 2]
        assert thetas.shape == theta_dots.shape == acts.shape

    acts = np.clip(acts, -2, 2)
    costs = angle_normalize(thetas) ** 2 + .1 * theta_dots ** 2 + .001 * (acts ** 2)

    return -costs

#### ダイナミクスモデルの実装

続いて、オンラインで学習するダイナミクスモデル $f_\theta$ を実装します。
ダイナミクスモデルはどのような関数近似器で近似しても良いですが、今回は2層のMLP (Multi-layer Perceptron) を用いましょう。

一点注意が必要なのは、今回再現実装しようとしている[論文](https://arxiv.org/abs/1708.02596)では、直接次の状態を予測するのではなく、**次の状態と現在の状態との差分**を予測します。つまり、下記損失関数を最小化するようなパラメータ $\theta$ を学習します。

$
\mathcal{E}(\theta)=\frac{1}{|\mathcal{D}|} \sum_{\left(\mathbf{s}_{t}, \mathbf{a}_{t}, \mathbf{s}_{t+1}\right) \in \mathcal{D}} \frac{1}{2}\left\|\left(\mathbf{s}_{t+1}-\mathbf{s}_{t}\right)-f_{\theta}\left(\mathbf{s}_{t}, \mathbf{a}_{t}\right)\right\|^{2}
$

ここで、$\mathcal{D}$ は遷移を保存したバッファとなります。
このような次の状態を直接予測するのではなく、その差分だけを予測する手法はよく使われるテクニックです。
これにより出力の分散をある程度抑えられるなどのメリットがあります。

それでは、実装していきましょう。

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Dense

class DynamicsModel(tf.keras.Model):
    def __init__(self, input_dim, output_dim, units=(32, 32)):
        super().__init__(name="MLP")

        self.l1 = Dense(units[0], name="L1", activation="relu")
        self.l2 = Dense(units[1], name="L2", activation="relu")
        self.l3 = Dense(output_dim, name="L3", activation="linear")

        self._optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

        with tf.device("/gpu:0"):
            self(tf.constant(np.zeros(shape=(1, input_dim), dtype=np.float32)))

    @tf.function
    def call(self, inputs):
        features = self.l1(inputs)
        features = self.l2(features)
        return self.l3(features)

    def predict(self, inputs):
        assert isinstance(inputs, np.ndarray)
        assert inputs.ndim == 2

        with tf.device("/gpu:0"):
            outputs = self.call(inputs)

        return outputs.numpy()

    @tf.function
    def fit(self, inputs, labels):
        with tf.GradientTape() as tape:
            predicts = self(inputs)
            loss = 0.5 * tf.reduce_mean(tf.square(labels - predicts))
        grads = tape.gradient(loss, self.trainable_variables)
        self._optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return loss

obs_dim = env.observation_space.high.size
act_dim = env.action_space.high.size

dynamics_model = DynamicsModel(input_dim=obs_dim + act_dim, output_dim=obs_dim)


def predict_next_state(obses, acts):
    assert obses.shape[0] == acts.shape[0]
    obs_diffs = dynamics_model.predict(np.concatenate([obses, acts], axis=1))
    assert obses.shape == obs_diffs.shape
    next_obses = obses + obs_diffs
    return next_obses

また、ダイナミクスモデルを学習する際に遷移を保存しておくバッファを実装します。
リングバッファを自分で実装しても良いですが、今回は既存のライブラリである[cpprb](https://github.com/ymd-h/cpprb)を用います。
バッファサイズは10Kとしてみます。
cpprb は Prioritized Replay Buffer や N-step Replay Buffer やそれらの組合せ、また画像入力タスクのためのフレームスタックなどもサポートしているので興味のある方は試してみてください。

In [ ]:
from cpprb import ReplayBuffer

# 10Kデータ分 (s, a, s') が保存できるリングバッファを用意します
rb_dict = {
    "size": 10000,
    "default_dtype": np.float32,
    "env_dict": {
        "obs": {"shape": env.observation_space.shape},
        "next_obs": {"shape": env.observation_space.shape},
        "act": {"shape": env.action_space.shape}}}
dynamics_buffer = ReplayBuffer(**rb_dict)

#### RS (Random-sample Shooting) の実装

次にRSを実装します。便利なので方策を用意しましょう。

In [ ]:
class RandomPolicy:
    def __init__(self, max_action, act_dim):
        self._max_action = max_action  # action の最大値
        self._act_dim = act_dim  # action の次元数

    def get_actions(self, batch_size):
        # 一様分布からバッチサイズ分ランダムにサンプリング
        return np.random.uniform(
            low=-self._max_action,
            high=self._max_action,
            size=(batch_size, self._act_dim))

policy = RandomPolicy(
    max_action=env.action_space.high[0],
    act_dim=env.action_space.high.size)


def random_shooting(init_obs, n_mpc_episodes=64, horizon=20):
    init_actions = policy.get_actions(batch_size=n_mpc_episodes)
    total_rewards = np.zeros(shape=(n_mpc_episodes,))
    obses = np.tile(init_obs, (n_mpc_episodes, 1))

    for i in range(horizon):
        acts = init_actions if i == 0 else policy.get_actions(batch_size=n_mpc_episodes)
        next_obses = predict_next_state(obses, acts)
        rewards = reward_fn(obses, acts)
        total_rewards += rewards
        obses = next_obses

    idx = np.argmax(total_rewards)
    return init_actions[idx]

#### 実行スクリプト

さて、いよいよRSを実行するコードを実装します。
冒頭の「全体の流れ」をコメントで繰り返しながら記述していきます。

In [ ]:
batch_size = 100
n_episodes = 100


# ランダムに遷移を収集し状態遷移モデルを収集します
for _ in range(10):
    obs = env.reset()
    for _ in range(200):
        act = env.action_space.sample()
        next_obs, _, done, _ = env.step(act)
        dynamics_buffer.add(obs=obs, act=act, next_obs=next_obs)
        obs = next_obs
        if done:
            break


def fit_dynamics(n_iter=50):
    mean_loss = 0.
    for _ in range(n_iter):
        samples = dynamics_buffer.sample(batch_size)
        inputs = np.concatenate([samples["obs"], samples["act"]], axis=1)
        labels = samples["next_obs"] - samples["obs"]
        mean_loss += dynamics_model.fit(inputs, labels).numpy()
    return mean_loss

# ダイナミクスモデルを事前学習します
fit_dynamics(n_iter=1000)


total_steps = 0
for episode_idx in range(n_episodes):
    total_rew = 0.

    obs = env.reset()
    for _ in range(episode_max_steps):
        total_steps += 1
        act = random_shooting(obs)
        next_obs, rew, done, _ = env.step(act)
        dynamics_buffer.add(
            obs=obs, act=act, next_obs=next_obs)
        total_rew += rew
        if done:
            break
        obs = next_obs

    mean_loss = fit_dynamics(n_iter=100)
    if episode_idx % 5 == 0:
        print("iter={0: 3d} total reward: {1: 4.4f} mean loss: {2:.6f}".format(episode_idx, total_rew, mean_loss))

最後に、学習済みモデルを使って振り子の動きを可視化してみましょう。

In [ ]:
# 学習済みモデルでの結果が見たいので過去のビデオリストを削除します
monitor_env.display(reset=True)

for episode_idx in range(3):
    obs = monitor_env.reset()
    total_rew = 0.
    for _ in range(episode_max_steps):
        act = random_shooting(obs)
        next_obs, rew, done, _ = monitor_env.step(act)
        total_rew += rew
        if done:
            break
        obs = next_obs
    print("iter={0: 3d} total reward: {1: 4.4f}".format(episode_idx, total_rew))

monitor_env.display()

学習済みモデルで振り子の振り上げがきちんとできていることが確認できましたか？
ただ、ご覧のように性能がイマイチですね。RSは実装が容易で、少ないサンプル数で解けることがメリットとして挙げられますが、例えば次のような欠点があります。

- $M$ステップ分の累積報酬和が最も良かったエピソードの**最初の**行動を選択するが、それが良いとは限らない（最初のステップは全然ダメでも2ステップ目以降が良い場合も考えられます）
- 先読みステップ数、エピソード数の設計にドメイン知識が必要

## アンサンブルダイナミクスモデルを用いた強化学習
モデルベースRLの利点・欠点をいくつかまとめましょう。
- 利点
  - サンプル効率が良い（サンプル効率：同じ性能に達するために必要な環境との相互作用数。少ない方が良い）
  - ダイナミクスモデルがわかっているので既存のプラニングアルゴリズムが使える
- 欠点
  - モデル誤差が大きい（ダイナミクスモデルが十分に実際の環境を近似できていない）と性能が出ない

モデルベースRLでは、現状手に入っているダイナミクスモデルを用いて問題を解きますが、モデル誤差が大きいとバイアス（真の環境との誤差）が積み重なり、最終的な性能が悪くなってしまいます。

この性質は逐次的な意思決定を行う強化学習では大きな問題となります。
2つ目の演習では、この欠点に挑戦します。
モデル誤差低減の方法はいくつかありますが、ここでは複数のモデルを同時に用いる手法により解決を試みます。
より具体的には、関数近似器を複数用意し、それらを効果的に用いることで方策の性能を向上させます。

### ME-TRPO

本演習では、下記論文について取り扱います。
- [Model-Ensemble Trust-Region Policy Optimization (ICLR2018)](https://arxiv.org/abs/1802.10592)

ME-TRPOでは、学習したダイナミクスモデルを**強化学習**に用います。
具体的には、ダイナミクスモデルで**擬似的な**遷移を生成し、それを用いて方策と価値関数を最適化します。
特徴を挙げておきます。

- 実環境で収集したサンプルを用いて**複数**のダイナミクスモデルを学習する
- **ダイナミクスモデルで生成したサンプル**を用いて方策を学習する

これにより、全体として必要な実環境との相互作用数を削減することを狙いとしています。

### 全体の流れ
以上のような特徴を念頭に置き、全体の流れを下記にまとめます。

1. 方策 $\pi_\theta$、ダイナミクスモデル $f_\phi$ の初期化
1. repeat
    1. 実環境 $f$ で $\pi_\theta$ を使ってサンプル収集
    1. ダイナミクスモデル $f_\phi$ の更新
    1. repeat
        1. $\pi_\theta$ と $f_\phi$ を使って遷移データを生成
        1. 生成された遷移データを使って $\pi_\theta$ を更新
        1. 方策評価 $\eta(\theta; \phi)$
    1. until 方策評価が高い限り
1. until 終了条件を満たすまで

それでは実装していきましょう。

### 準備
ME-TRPOのコア部分の実装の前に、他に必要な機能について準備します。


#### Pendulum環境の設定

論文の設定に近くなるように環境側の設定を少し見直します。

In [ ]:
episode_max_steps = 100
debug = False

#### ダイナミクスモデル

ダイナミクスモデルを**複数**定義します。
また、ダイナミクスモデル学習用のバッファを初期化しておきます。

In [ ]:
# ダイナミクスモデルを5つ用います
n_dynamics_model = 5

obs_dim = env.observation_space.high.size
act_dim = env.action_space.high.size

dynamics_models = [DynamicsModel(
    input_dim=obs_dim + act_dim, output_dim=obs_dim) for _ in range(n_dynamics_model)]


def predict_next_state(obses, acts, idx=None):
    is_single_input = obses.ndim == acts.ndim and acts.ndim == 1
    if is_single_input:
        obses = np.expand_dims(obses, axis=0)
        acts = np.expand_dims(acts, axis=0)

    inputs = np.concatenate([obses, acts], axis=1)
    idx = np.random.randint(n_dynamics_model) if idx is None else idx
    obs_diffs = dynamics_models[idx].predict(inputs)

    if is_single_input:
        return (obses + obs_diffs)[0]
    return obses + obs_diffs

#### 方策

[論文](https://arxiv.org/abs/1802.10592)では[TRPO](https://arxiv.org/abs/1502.05477)を用いましたが、やや複雑なので代わりに[PPO](https://arxiv.org/abs/1707.06347)を用います。

In [ ]:
from tf2rl.algos.ppo import PPO

def define_policy():
    return PPO(
        state_shape=env.observation_space.shape,
        action_dim=env.action_space.shape[0],
        max_action=env.action_space.high[0],
        is_discrete=False,
        batch_size=64,
        actor_units=(32, 32),
        critic_units=(32, 32),
        n_epoch=10,
        lr_actor=3e-4,
        lr_critic=3e-4,
        hidden_activation_actor="tanh",
        hidden_activation_critic="tanh",
        discount=0.9,
        lam=0.95,
        entropy_coef=0.,
        horizon=2048,
        normalize_adv=True,
        enable_gae=True,
        gpu=0)

policy = define_policy()

def clip_action(action):
    return np.clip(action, env.action_space.low, env.action_space.high)

また、方策学習用のデータを格納するバッファを用意します。

In [ ]:
rb_dict = {
    "size": policy.horizon,
    "default_dtype": np.float32,
    "env_dict": {
        "obs": {"shape": env.observation_space.shape},
        "act": {"shape": env.action_space.shape},
        "done": {},
        "logp": {},
        "ret": {},
        "adv": {}}}
on_policy_buffer = ReplayBuffer(**rb_dict)

rb_dict = {
    "size": episode_max_steps,
    "default_dtype": np.float32,
    "env_dict": {
        "obs": {"shape": env.observation_space.shape},
        "act": {"shape": env.action_space.shape},
        "next_obs": {"shape": env.observation_space.shape},
        "rew": {},
        "done": {},
        "logp": {},
        "val": {}}}
episode_buffer = ReplayBuffer(**rb_dict)

方策を評価するためのコードも用意しておきます。

In [ ]:
def evaluate_policy(total_steps, test_episodes=10):
    avg_test_return = 0.
    for i in range(test_episodes):
        episode_return = 0.
        obs = env.reset()
        for _ in range(episode_max_steps):
            act, _ = policy.get_action(obs, test=True)
            next_obs, rew, _, _ = env.step(clip_action(act))
            episode_return += rew
            obs = next_obs
        avg_test_return += episode_return
    return avg_test_return / test_episodes

### 実環境 $f$ での $\pi_\theta$ を使って遷移データを生成


In [ ]:
def collect_transitions_real_env():
    obs = env.reset()
    episode_steps = 0
    for _ in range(policy.horizon):
        episode_steps += 1
        act, _ = policy.get_action(obs)
        next_obs, *_ = env.step(clip_action(act))
        dynamics_buffer.add(obs=obs, act=act, next_obs=next_obs)
        obs = next_obs
        if episode_steps == episode_max_steps:
            episode_steps = 0
            obs = env.reset()

### ダイナミクスモデルの更新


In [ ]:
def fit_dynamics(n_iter=50):
    mean_losses = np.zeros(shape=(n_dynamics_model,), dtype=np.float32)
    for _ in range(n_iter):
        samples = dynamics_buffer.sample(batch_size)
        inputs = np.concatenate([samples["obs"], samples["act"]], axis=1)
        labels = samples["next_obs"] - samples["obs"]
        for i, dynamics_model in enumerate(dynamics_models):
            mean_losses[i] += dynamics_model.fit(inputs, labels).numpy()
    return mean_losses


### $\pi_\theta$ と $f_\phi$ を使って遷移データを生成


In [ ]:
from tf2rl.misc.discount_cumsum import discount_cumsum


def collect_transitions_sim_env():
    """
    Generate transitions using dynamics model
    :return:
    """
    on_policy_buffer.clear()
    n_episodes = 0
    ave_episode_return = 0
    while on_policy_buffer.get_stored_size() < policy.horizon:
        obs = env.reset()
        episode_return = 0.
        for i in range(episode_max_steps):
            act, logp, val = policy.get_action_and_val(obs)
            env_act = clip_action(act)
            if debug:
                next_obs, rew, _, _ = env.step(env_act)
            else:
                next_obs = predict_next_state(obs, env_act)
                rew = reward_fn(obs, act)[0]
            episode_buffer.add(obs=obs, act=act, next_obs=next_obs, rew=rew,
                               done=False, logp=logp, val=val)
            obs = next_obs
            episode_return += rew
        finish_horizon(last_val=val)
        ave_episode_return += episode_return
        n_episodes += 1
    return ave_episode_return / n_episodes


def finish_horizon(last_val=0):
    samples = episode_buffer._encode_sample(np.arange(episode_buffer.get_stored_size()))
    rews = np.append(samples["rew"], last_val)
    vals = np.append(samples["val"], last_val)

    # GAE-Lambda advantage calculation
    deltas = rews[:-1] + policy.discount * vals[1:] - vals[:-1]
    advs = discount_cumsum(deltas, policy.discount * policy.lam)

    # Rewards-to-go, to be targets for the value function
    rets = discount_cumsum(rews, policy.discount)[:-1]
    on_policy_buffer.add(
        obs=samples["obs"], act=samples["act"], done=samples["done"],
        ret=rets, adv=advs, logp=np.squeeze(samples["logp"]))
    episode_buffer.clear()

### 生成された遷移データを使って $\pi_\theta$ を更新



In [ ]:
def update_policy():
    # Compute mean and std for normalizing advantage
    samples = on_policy_buffer.get_all_transitions()
    mean_adv = np.mean(samples["adv"])
    std_adv = np.std(samples["adv"])

    for _ in range(policy.n_epoch):
        samples = on_policy_buffer._encode_sample(np.random.permutation(policy.horizon))
        adv = (samples["adv"] - mean_adv) / (std_adv + 1e-8)
        actor_loss, critic_loss = 0., 0.
        for idx in range(int(policy.horizon / policy.batch_size)):
            target = slice(idx * policy.batch_size, (idx + 1) * policy.batch_size)
            losses = policy.train(
                states=samples["obs"][target],
                actions=samples["act"][target],
                advantages=adv[target],
                logp_olds=samples["logp"][target],
                returns=samples["ret"][target])
            actor_loss += losses[0]
            critic_loss += losses[0]

### 方策評価 $\eta(\theta; \phi)$


In [ ]:
# 各ダイナミクスモデルにおいて何エピソード分評価するかを決めます
n_eval_episodes_per_model = 5

def evaluate_current_return(init_states):
    n_episodes = n_dynamics_model * n_eval_episodes_per_model
    assert init_states.shape[0] == n_episodes

    obses = init_states.copy()
    next_obses = np.zeros_like(obses)
    returns = np.zeros(shape=(n_episodes,), dtype=np.float32)

    for _ in range(episode_max_steps):
        acts, _ = policy.get_action(obses)
        for i in range(n_episodes):
            model_idx = i // n_eval_episodes_per_model
            env_act = np.clip(acts[i], env.action_space.low, env.action_space.high)
            next_obses[i] = predict_next_state(obses[i], env_act, idx=model_idx)
        returns += reward_fn(obses, acts)
        obses = next_obses

    return returns

### 実行

以上で必要なパーツは全て揃いました。

In [ ]:
total_steps = 0
test_episodes = 10

while True:
    # Collect samples
    collect_transitions_real_env()
    total_steps += policy.horizon

    # Fit dynamics models
    fit_dynamics()

    # ret_real_env, ret_sim_env = self._evaluate_model()
    # print("Returns (real, sim) = ({: .3f}, {: .3f})".format(ret_real_env, ret_sim_env))

    n_updates = 0
    init_states_for_eval = np.array([
        env.reset() for _ in range(n_dynamics_model * n_eval_episodes_per_model)])
    returns_before_update = evaluate_current_return(init_states_for_eval)
    while True:
        n_updates += 1

        # Generate samples using dynamics models
        average_return = collect_transitions_sim_env()

        # Update policy
        update_policy()

        returns_after_update = evaluate_current_return(init_states_for_eval)
        improved_ratio = np.sum(returns_after_update > returns_before_update) / (
                    n_dynamics_model * n_eval_episodes_per_model)
        print("Training total steps: {0: 7} improved ratio: {1: .2f} simulated return: {2: .4f} at n_update: {3: 2}".format(
            total_steps, improved_ratio, average_return, n_updates))
        if improved_ratio < 0.7:
            break
        returns_before_update = returns_after_update

    # Evaluate policy
    if total_steps // policy.horizon % 10 == 0:
        avg_test_return = evaluate_policy(total_steps, test_episodes)
        print("Evaluation Total Steps: {0: 7} Average Reward {1: 5.4f} over {2: 2} episodes".format(
            total_steps, avg_test_return, test_episodes))